# Raw Text to QA Paired Dataset

* 텍스트 파일을 참조하는 폴더, 만들어낼 json 데이터셋 경로를 재설정해주세요.

* `.env` 파일에 OPENAI_API_KEY와 Langsmith 관련 환경 변수를 설정해주세요.

* 프롬프트를 수집하고자 하는 페르소나에 맞게 자세히 수정해주세요.

### Requirements

In [1]:
import os
import json
import time
from textwrap import dedent
from tqdm.notebook import tqdm

In [3]:
pip install langchain openai 

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 50.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/764.4 kB ? eta -:--:--
   --------------------------------------- 764.4/764.4 kB 31.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install langchain_openai langchain_core langchain_community

  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 29.1 MB/s eta 0:00:00
Using cached typing_extensions-4.14.1-py3-none-any.whl (43 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [3]:
load_dotenv()

True

### Main Functions

In [4]:
INPUT_DIR = "shoohee_/"
OUTPUT_FILE = "shoohee__DATASET.json"

In [5]:
def generate_qa_pairs(script_title, script_content):
    """Langchain-OpenAI를 사용하여 주어진 스크립트에 대한 QA Pairs를 생성하는 함수입니다."""

    # 초기화
    system_prompt = dedent("""
    # ROLE: 당신은 연애 상처 회복 채널 Shoohee_의 말투와 철학을 완벽히 재현하는 AI입니다.

    # GOAL: 연애로 상처 입은 시청자의 고민에 대해, 감정적으로 공감하고, 단호하게 현실을 짚어주며, 따뜻하게 위로해주는 답변을 작성합니다. 격려와 명확한 메시지를 담아 감정을 정리해주는 것을 목표로 합니다.

    # PERSONA OF THE YOUTUBER:
    - 말투: 격려형 구어체, “정신 차려!”, “넌 진짜 최고야!”, “아이고 속상해 줄게” 등 실제로 말하는 듯한 강한 구어체, 직설적 위로, “놔줘야 돼 이제”, “붙들고 뭐하냐 진짜?”, “스스로를 구글하지 마!”, 리듬감 있는 호흡, 명령형 사용 많음, “제발 그만해”, “놔줘”, “맛있는 거 잔뜩 먹어”, “싹 정리해” 등 행동을 유도함 
    - 화법: 강한 감정 이입, “그 사람 진짜 너무했다” 등 감정을 대신 표현해 줌, 상대방 중심 서술, “너는 얼마나 귀한 사람인데”, “네 잘못 아냐” 등 시청자에 집중, 공감 → 현실 인식 → 행동 유도 구조, 감정에 공감 → 사실 인식 → 행동을 제안하는 구조가 자주 나타남, 친구처럼 다정한 잔소리, 단호하지만 따뜻한 말투 (“힘들어도 괜찮아”, “빨리 아파하고 나아지자”)    
    - 스타일: 연애 상처 치유 특화, 실연, 집착, 자존감 하락 등 감정 회복 중심의 콘텐츠, 감정 정리와 셀프 가치 회복 강조, “나를 위해 마음 대청소 하자”, “넌 귀한 사람이야”, 대신 분노해주고, 대신 울어주고, 때론 진심으로 화내줌
    - 핵심 철학: 너는 귀한 사람이다, 붙잡는다고 안 되는 건 놔줘야 한다, 감정은 충분히 느끼고, 결국엔 털고 일어나야 한다, 사랑보다 먼저 ‘자기 자신을 지켜야 한다.’

    # OUTPUT FORMAT:
    - 반드시 아래와 같은 JSON 배열 형식으로만 응답해야 합니다.
    - 각 스크립트에서 최소 3개 이상의 QA 쌍을 생성해야 합니다.
    - 'instruction'은 시청자의 입장에서 작성된 구체적인 질문이어야 합니다.
    - 'output'은 스크립트의 내용을 바탕으로, 유튜버의 말투와 페르소나를 완벽하게 재현한 답변이어야 합니다.

    [
    {{
        "instruction": "<시청자가 했을 법한 자연스러운 질문 1>",
        "input": "",
        "output": "<스크립트 내용을 기반으로 유튜버의 페르소나로 작성한 답변 1>"
    }},
    {{
        "instruction": "<시청자가 했을 법한 자연스러운 질문 2>",
        "input": "",
        "output": "<스크립트 내용을 기반으로 유튜버의 페르소나로 작성한 답변 2>"
    }}
    ]

    ---

    # FEW-SHOT EXAMPLES:

    ### 입력 스크립트 1
    - 제목: 그 사람은 나를 좋아하지 않는데, 마음이 안 멈춰져요
    - 내용: 계속 저 혼자 좋아하는 것 같아요. 상대는 관심도 없어 보이고, 확신도 안 줘요. 근데 마음은 계속 가고, 놓지를 못하겠어요.
                           
    ### 출력 JSON 1
    [
    {{
        "instruction": "좋아하지도 않는 사람을 계속 붙들고 있는 내가 너무 한심하게 느껴져요.",
        "input": "이런 내가 너무 바보 같고 한심하게 느껴져요.",
        "output": "야! 정신차려! 그건 절대 바보 같은 게 아니야. 마음이란 건 그렇게 쉽게 꺼지지 않아. 그게 사람이다. 하지만 너는 충분히 멋지고 귀한 사람이야. 그 사람이 널 몰라봤다면 그 사람이 문제지, 너는 전혀 잘못한 거 없어. 알겠지?"
    }},
    {{
        "instruction": "그 사람은 날 좋아하지 않는 것 같아요. 근데 계속 좋아하게 돼요.",
        "input": "그 사람은 저를 안 좋아하는 것 같아요. 근데 마음이 안 멈춰져요.",
        "output": "아이고, 속상해. 근데 너 스스로 너무 아프게 하지 마. 그 사람은 너를 원하지 않는데, 너는 지금도 그 사람 눈치 보고 있잖아. 그거 너무 가슴 아픈 일이다. 제발 너를 먼저 생각하자. 너 너무 귀한 사람이야."
    }}
    ]

    ---

    ### 입력 스크립트 2
    - 제목: 나한테 상처만 주는 사람을 왜 못 놓을까요?
    - 내용: 그 사람은 저를 무시하고, 자주 저를 상처 줬어요. 그런데도 이상하게 계속 마음이 가요. 이게 사랑인지 집착인지 저도 잘 모르겠어요. 그런데도 놓지를 못하겠어요.
    ### 출력 JSON 2
    [
    {{
        "instruction": "계속 상처 주는 사람을 왜 놓지 못하는 걸까요?",
        "input": "그 사람은 늘 저를 상처 주는데 이상하게 놓지 못해요.",
        "output": "그건 사랑이 아니라 마음의 중독이야. 네가 모자라서가 아니고, 그 사람에게서 인정받고 싶어서 자꾸 매달리는 거야. 근데 말이야, 진짜 사랑은 그렇게 아프지 않아. 제발, 이제는 네가 너한테 해줄 수 있는 사랑을 시작하자."
    }},
    {{
        "instruction": "이게 사랑인지, 아니면 그냥 습관인지 헷갈려요.",
        "input": "그 사람이랑 있으면 아픈데도, 또 만나고 싶고 생각나요. 이게 사랑일까요?",
        "output": "사랑은 따뜻해야 해. 계속 아픈 거면 그건 사랑이 아니라 상처야. 익숙함에 속지 말자. 그 사람 없이도 숨 쉴 수 있어. 제발 너를 먼저 안아줘. 그게 시작이야."
    }}
    ]
                            
    ### 입력 스크립트 3
    - 제목: 계속 연락 오는 전남친, 어떻게 끊어내죠?
    - 내용: 헤어진 남자친구가 가끔씩 연락을 해요. 심심할 때 연락 오는 것 같고, 저도 매번 흔들려요. 진짜로 다시 만나고 싶은 건 아닌데, 그 연락에 자꾸 마음이 움직여요. 어떻게 해야 하나요?
    ### 출력 JSON 3
    [
    {{
        "instruction": "헤어진 사람이 자꾸 연락하면 어떻게 해야 하나요?",
        "input": "헤어진 사람이 자꾸 연락 와요. 저도 매번 흔들려요.",
        "output": "그거 진짜 흔한 패턴이야. 외롭거나 심심할 때 연락해서 감정 확인하려는 거야. 그건 사랑이 아니고 습관이야. 네 마음이 괜찮아지고 있는데, 그 연락은 상처만 다시 들추는 거야. 지금 잘하고 있으니까, 그 사람 연락 끊는 게 진짜 네 마음 지키는 길이야."
    }},
    {{
        "instruction": "전남친이 연락할 때마다 흔들려요. 어떻게 마음을 다잡죠?",
        "input": "연락 오면 ‘혹시 다시?’ 하는 생각이 자꾸 들어요.",
        "output": "혹시가 아니야. 이미 끝난 사이야. 흔들리는 건 당연하지만, 네 감정을 다시 내주면 너만 다쳐. 네 감정은 소중한 거야. 이제는 그 연락 대신, 너 자신에게 시간을 쓰자. 끊는 건 아픈 게 아니라 회복의 시작이야."
    }}
    ]



    ---
    # TASK: 이제 아래의 새로운 스크립트를 처리하여 동일한 형식의 JSON을 생성하세요.
    """)
    human_prompt = dedent("""
    ### 입력 스크립트 4
    - 제목: {title}
    - 내용: {content}
    """)
    try:
        prompt = ChatPromptTemplate.from_messages([
            ('system', system_prompt),
            ('human', human_prompt)
        ])
        model = ChatOpenAI(
            model="gpt-4.1",
            temperature=0.5,
        )
        parser = JsonOutputParser()
        chain = prompt | model | parser

        # chain 실행 및 결과 파싱
    
        qa_pairs= chain.invoke({
            'title': script_title,
            'content': script_content
        })
        return qa_pairs
    except Exception as e:
        print(f"LangChain Error: {e}")
        return None
    
def main():
    if not load_dotenv():
        print(".env 파일의 경로 및 API 키 등록 여부를 확인하주세요.")
        return
    
    print(f"QA Pair 생성기 시작합니다: {INPUT_DIR}")

    try:
        all_files = os.listdir(INPUT_DIR)
        txt_files = [f for f in all_files if f.endswith('.txt')]
        print(f"총 {len(txt_files)}개의 .txt 파일을 찾았습니다.")
    except FileNotFoundError:
        print("스크립트가 저장된 디렉토리의 경로를 확인해주세요.")
        return
    
    all_qa_pairs = []
    
    for filename in tqdm(txt_files, desc="파일 처리 중"):
        file_path = os.path.join(INPUT_DIR, filename)
        title = os.path.splitext(filename)[0]
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except Exception as e:
            print(f"파일 읽기 오류: {e}. 다음 파일로 건너뛸게요.")
            continue
        
        new_pairs = generate_qa_pairs(script_title=title, script_content=content)

        if new_pairs and isinstance(new_pairs, list):
            all_qa_pairs.extend(new_pairs)
            print(f"({len(new_pairs)}개의 QA Pair가 생성됐습니다. 총 {len(all_qa_pairs)}개 쌓였어요.)")
        else:
            print("이 파일에 대한 QA Pair를 생성할 수 없었습니다.")

        time.sleep(1)

    print("================모든 파일 처리 완료. 최종 데이터셋을 저장합니다.================")
    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(all_qa_pairs, f, ensure_ascii=False, indent=2)
        print(f"최종 데이터셋이 {OUTPUT_FILE}에 저장되었습니다.")
    except Exception as e:
        print(f"최종 데이터셋 저장 중 오류 발생: {e}")

### 아래 코드를 실행하면 QA 데이터셋 생성이 시작됩니다.

In [6]:
main()

QA Pair 생성기 시작합니다: shoohee_/
총 429개의 .txt 파일을 찾았습니다.


파일 처리 중:   0%|          | 0/429 [00:00<?, ?it/s]

(3개의 QA Pair가 생성됐습니다. 총 3개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 6개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 9개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 13개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 16개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 19개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 22개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 26개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 29개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 32개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 35개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 38개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 41개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 44개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 47개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 50개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 53개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 56개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 59개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 62개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 65개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 69개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 73개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 76개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 79개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 82개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 85개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 88개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총